In [7]:
import chess.pgn
import pandas as pd
from tqdm import tqdm
import io
import time

In [8]:
def get_eval(game,n = None, t = None):
    # Note - I would like to make a class which contains all important metrics for my neural net
    board = game.board()
    move_number = 0
    white = []
    black = []
    centipawns = []
    engine = chess.engine.SimpleEngine.popen_uci("stockfish_15_win_x64_avx2\\stockfish_15_x64_avx2.exe")
    engine.configure({"Threads": 8}) # or even higher

    for move in game.mainline_moves():
        board.push(move)
        if move_number == 0:
            info = engine.analyse(board, chess.engine.Limit(depth = n,time=t))
            prev_centipawn = info["score"].white().score()
        if move_number > 0:
            info = engine.analyse(board, chess.engine.Limit(depth = n,time = t))
            centipawn = info["score"].white().score()
            if centipawn is None:
                centipawn = 0
            if move_number % 2 == 0:
                white.append(centipawn - prev_centipawn)
            else:
                black.append(- (centipawn - prev_centipawn))
            prev_centipawn = centipawn
            centipawns.append(centipawn)
        move_number +=1
    engine.quit()
    return white, black, centipawns

In [9]:
class Eval:
    def __init__(self, game, n):
        self.game = game
        self.white_eval,self.black_eval,self.centipawns  = get_eval(self.game, n)

In [10]:
def to_eval(x,n):
    eval = Eval(x,n)
    return eval

In [11]:
def pgn_to_csv(player):
    i = 0
    file2 = open('pgns/' +player + '.pgn',encoding="utf-8",errors = "ignore")

    #TODO - change this to a more standard while loop
    game = chess.pgn.read_game(file2)
    df = pd.read_csv('blitz/' + player +'.csv')
    df = df[["White", "Black", "WhiteELO", "BlackELO", "Date","Event", "Result", "Opening", "Rounds", "Game", "Eval", "EventType"]]
    j = len(df.index)
    while isinstance(game,chess.pgn.Game) and i < 750:
        white = []
        black = []
        white_elo = []
        black_elo = []
        date = []
        event = []
        result = []
        wdiff = []
        bdiff = []
        rounds = []
        opening = []
        games = []
        evals = []
        i +=1
        if i <= j:
            game = chess.pgn.read_game(file2)
            continue
        start = time.time()
        print(i)
        if "White" in game.headers:
            white.append(game.headers["White"])
        else:
            white.append('*')
        if "Round" in game.headers:
            rounds.append(game.headers["Round"])
        else:
            white.append('*')
        if "Black" in game.headers:
            black.append(game.headers["Black"])
        else:
            black.append('*')
        if "WhiteElo" in game.headers:
            white_elo.append(game.headers["WhiteElo"])
        else:
            white_elo.append('*')
        if "BlackElo" in game.headers:
            black_elo.append(game.headers["BlackElo"])
        else:
            black_elo.append('*')
        if "Date" in game.headers:
            date.append(game.headers["Date"])
        else:
            date.append('*')
        if "EventType" in game.headers:
            event.append(game.headers["EventType"])
            if 'blitz' in game.headers["EventType"]:
                evals.append(to_eval(game ,20).centipawns)
            else:
                evals.append('')
        else:
            event.append('*')
            evals.append('')
        if "ECO" in game.headers:
            opening.append(game.headers["ECO"])
        else:
            opening.append('*')
        if "Result" in game.headers:
            result.append(game.headers["Result"])
        else:
            result.append('*')
        games.append(game.mainline_moves())
        #evals.append(to_eval(game ,20).centipawns)
        end = time.time()
        print(f"time elapsed: {end - start}")
        game = chess.pgn.read_game(file2)

        data = {"White": white, "Black": black, "WhiteELO": white_elo, "BlackELO": black_elo, "Date": date,"Event": event, "Result": result, "Opening": opening, "Rounds": rounds, "Game": games, "Eval": evals, "EventType": event}

        df1 = pd.DataFrame(data)

        df = pd.concat([df,df1])


        df = df.reset_index(drop=True)

        df.to_csv('blitz/' + player +'.csv')

In [ ]:
players = ['andreikin, dmitry', 'anand, viswanathan', 'wang, hao', 'grischuk, alexander', 'karjakin, sergey','duda, jan-krzysztof', 'radjabov, teimour', 'dominguez perez, leinier','nakamura, hikaru', 'vachier-lagrave, maxime','aronian, levon','mamedyarov, shakhriyar', 'so, wesley','ding, liren','carlsen, magnus', 'rapport, richard', 'nepomniachtchi, ian', 'giri, anish', 'firouzja, alireza', 'caruana, fabiano','zelcic, robert','khotenashvili, bela', 'bischoff, klaus', 'hoffmann, asa','kaufman, lawrence','bellaiche, elise']
players.reverse()
for player in players:
    print(player)
    pgn_to_csv(player)

bellaiche, elise
kaufman, lawrence
525
time elapsed: 84.94222354888916
526
time elapsed: 0.0
527
time elapsed: 1.5735626220703125e-05
528
time elapsed: 0.0
529
time elapsed: 0.0
530
time elapsed: 0.0005197525024414062
531
time elapsed: 0.0
532
time elapsed: 0.0
533
time elapsed: 0.0
534
time elapsed: 0.0
535
time elapsed: 0.0
536
time elapsed: 0.0
537
time elapsed: 0.0
538
time elapsed: 7.939338684082031e-05
539
time elapsed: 0.0
540
time elapsed: 0.0
541
time elapsed: 0.0
542
time elapsed: 0.0
543
time elapsed: 0.0
544
time elapsed: 97.30641031265259
545
time elapsed: 0.0005135536193847656
546
time elapsed: 0.0
547
time elapsed: 0.0
548
time elapsed: 0.0
549
time elapsed: 0.0
550
time elapsed: 0.0
551
time elapsed: 0.0
552
time elapsed: 0.0
553
time elapsed: 0.0
554
time elapsed: 0.0
555
time elapsed: 0.0
556
time elapsed: 0.0
557
time elapsed: 0.0
558
time elapsed: 0.0
559
time elapsed: 0.0
560
time elapsed: 0.0
561
time elapsed: 0.0
562
time elapsed: 216.86149644851685
563
time elap

In [ ]:
# data = {"White": [], "Black": [], "WhiteELO": [], "BlackELO": [], "Date": [],"Event": [], "Result": [], "Opening": [], "Rounds": [], "Game": [], "Eval": [], "EventType": []}
#
# df = pd.DataFrame(data)
# for player in players:
#     df.to_csv('blitz/' + player + '.csv')